In [ ]:
from collections import Counter, defaultdict
from copy import deepcopy

import nltk
import pandas as pd
import yaml

In [15]:
import numpy as np

In [16]:
next(iter([]),np.nan)

nan

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords

stopwords = set(stopwords.words("english"))

In [ ]:
def div_counter(counter1, counter2):
    quotient = deepcopy(counter1)
    for key in quotient:
        if key in counter2:
            quotient[key] = counter1[key] / counter2[key]
        else:
            quotient[key] = 9999
    return quotient

In [ ]:
df = pd.read_csv("/home/nlp/adarsh/soprano_nlp_postprocessing/resources/kb/tumor_2.1.csv")[
    ["name", "synonym"]
]

In [ ]:
keywords_df = (
    df.groupby("name")["synonym"]
    .agg(" ".join)
    .reset_index()
    .assign(synonym=lambda df: df["synonym"].str.split())
    .rename(columns={"synonym": "keywords", "name": "class"})
)
keyword_dict = keywords_df.set_index("class").to_dict()["keywords"]

In [ ]:
keyword_count_per_class = {}
for class_name, keywords in keyword_dict.items():
    rest = deepcopy(keyword_dict)
    rest.pop(class_name)
    rest_keyword_list = sum(rest.values(), [])
    keyword_counter = Counter(keywords)
    rest_keyword_counter = Counter(rest_keyword_list)
    diff = keyword_counter - rest_keyword_counter
    quotient = div_counter(keyword_counter, rest_keyword_counter)
    keyword_count_per_class[class_name] = {"diff": diff, "quotient": quotient}

In [ ]:
keywords_for_each_class = defaultdict(dict)
for class_name, diff_quotient in keyword_count_per_class.items():
    diff = diff_quotient["diff"]
    quotient = diff_quotient["quotient"]
    for word, word_diff in diff.items():
        if word_diff >= 2 and quotient[word] >= 10 and word not in stopwords and not word.isnumeric():
            keywords_for_each_class[class_name][word] = Counter(keyword_dict[class_name])[word]

In [ ]:
def fun():
    return defaultdict(set)


keywords_as_edge = defaultdict(fun)
exclusive_keywords_in_each_class = defaultdict(fun)

for class_name, phrases in (
    df.groupby("name")["synonym"].agg(list).reset_index().set_index("name").to_dict()["synonym"].items()
):
    for phrase in phrases:
        for keyword in sorted(
            keywords_for_each_class[class_name],
            key=lambda x: keywords_for_each_class[class_name][x],
            reverse=True,
        ):
            if keyword in phrase:
                keywords_as_edge[class_name][phrase].add(keyword)
                exclusive_keywords_in_each_class[class_name][keyword].add(phrase)
                # break

In [ ]:
edges_as_tuples = {
    class_name: [tuple(i) for i in value.values()] for class_name, value in keywords_as_edge.items()
}

In [ ]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
def calculate_diff(i):
    weight_per_column = np.sum(weight[weight[:, i] == 1].sum(axis=1) > 1)
    number_of_occurences = len(weight[weight[:, i] == 1].sum(axis=1))
    bonus = 1 if i not in visited else 0
    return number_of_occurences - weight_per_column + bonus

In [ ]:
regex_kb = defaultdict(list)
for key in edges_as_tuples:
    weight = mlb.fit_transform(edges_as_tuples[key])
    classes = list(mlb.classes_)
    visited = set()

    while True:
        diff = max([calculate_diff(i) for i in range(len(classes))])
        if diff == 0:
            break
        i = max(range(len(classes)), key=calculate_diff)
        visited.add(i)
        regex_kb[key].append(classes[i].replace("(", ""))
        weight[weight[:, i] == 1] = 0
        for j in np.arange(len(classes))[np.sum(weight, axis=0) == 0]:
            visited.add(j)

In [ ]:
dict(regex_kb)

In [ ]:
import pandas as pd
import polars as pl

In [ ]:
df_pl = pl.read_parquet("/home/nlp/adarsh/soprano_nlp_postprocessing/data/master_with_element.parquet")
df = pd.read_parquet("/home/nlp/adarsh/soprano_nlp_postprocessing/data/master_with_element.parquet")

In [ ]:
df_pl.drop("nlp_decision_flag").unique()

In [ ]:
df_pl = df_pl.lazy()

In [ ]:
# %%timeit
regex = [pl.col("context")] + [
    pl.col("context").str.contains("|".join(regex_kb[cancer])).alias(cancer) for cancer in regex_kb
]
(
    df_pl.unique(["context"])
    .select(regex)
    .melt(id_vars="context", value_vars=[cancer for cancer in regex_kb])
    .filter(pl.col("value"))
    .groupby("context")
    .agg(pl.col("variable").unique().alias("cancer_prediction"))
    .join(df_pl, on=["context"], how="inner")
    .unique(list(df_pl.columns))
    .explode("cancer_prediction")
    .filter(pl.col("cancer_prediction").is_not_null())
    # .collect()
)

In [ ]:
df

In [ ]:
%%timeit
for cancer in regex_kb:
    df[cancer] = df["prediction"].str.contains("|".join(regex_kb[cancer]))

In [ ]:
177 / 4.74

In [2]:
import pandas as pd

df = pd.read_excel("/Users/adarshgupta/Downloads/DocFinder Labels 082023.xlsx")

In [3]:
df.to_parquet("/Users/adarshgupta/projects/learn_rust/test.parquet")

In [3]:
with open("../data/comorbity.txt") as f:
    texts = f.readlines()
    texts = [text.strip() for text in texts if text.strip()]

In [22]:
from rapidfuzz.utils import default_process

texts = [" ".join(default_process(text).split())+"\n" for text in texts]

In [26]:
texts

['aids',
 'aids hiv infection opportunistic infection',
 'hiv infection symptomatic',
 'hiv patient with candidiasis of bronchi trachea esophagus or lungs',
 'hiv patient with coccidioidomycosis',
 'hiv patient with cryptococcosis',
 'hiv patient with cryptosporidiosis chronic intestinal greater than 1 month s duration',
 'hiv patient with cytomegalovirus disease particularly cmv retinitis',
 'hiv patient with encephalopahty hiv related',
 'hiv patient with herpes simplex bronchitis pneumonitis or esophagitis',
 'hiv patient with histoplasmosis',
 'hiv patient with invasive cervical cancer',
 'hiv patient with isosporiasis chronic intestinal',
 'hiv patient with kaposi s sarcoma',
 'hiv patient with lymphoma that is burkitt s type immunoblastic or that is primary and affects the brain or central nervous system',
 'hiv patient with mycobacterium avium infection',
 'hiv patient with pneumocystis carinii pneumonia',
 'hiv patient with pneumonia recurrent',
 'hiv patient with progressive m

In [23]:
with open("../data/comorbity.txt","w") as f:
    f.writelines(texts)

In [4]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

embeddings = model.encode(texts)



In [5]:
texts_emd = {text:emd for text,emd in zip(texts,embeddings)}

In [10]:
import numpy as np
source_text = "systolic"
source_emd = model.encode(source_text)
sorted(texts,key=lambda i :(texts_emd[i]@source_emd)/np.sqrt((texts_emd[i]@texts_emd[i]) * (source_emd@source_emd)),reverse=True )[:10]

['hypertensive heart disease with heart failure',
 'cardiomyopathy',
 'ischemic cardiomyopathy',
 'stroke',
 'rheumatic heart failure congestive',
 'stemi nstemi',
 'aortitis',
 'malignant hypertensive heart disease with heart failure',
 'atherosclerosis',
 'alcoholic cardiomyopathy']

In [ ]:
{' diverticulosis': 1,
'peripheral': 1,
'neuropathy': 1,
'neop]': 1,
'anxiety': 1,
'marrow': 1,
'tear': 1,
'urinary': 1,
'tendon': 1,
'hyperkalemia': 1,
'sinusitis': 1,
'trouble': 1,
'gall': 1,
'stones': 1,
'bcc': 1,
'reccurant': 1,
'arthropathies': 1,
'vitamin': 1,
'macrocytosis': 1,
'the': 1,
'disorder': 1,
'depression': 1,
'description': 1,
'vaginal': 1,
'illness': 1,
'adenosis': 1,
'sores': 1,
'illnesses': 1,
'spinal': 1,
'hypertension': 1,
'osteoarthritis': 1,
'prolapse': 1,
'systolic': 1,
'2/pelvic': 1,
}